In [ ]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="e6af334544b37a85e83900a9152eb9a0.GzjnBTUhPOhDDJhx")

template = """
请基于以下公告内容，进行以下分析，确定公告类型，是否为招标/资审公告（即是否包含基本招标信息，如项目名称、项目信息、基本资格要求和投标截止日期等。）

[在这里插入公告内容]
另附，可能的公告类型
- 招标/资审公告：正式邀请符合资格的投标人参与投标的公告，提供拟采购的货物、工程或服务的基本信息、投标条件等。
- 公告答疑：对潜在投标人提出的问题给出的官方回答。
- 公告变更：对先前发布的招标公告内容进行的正式更改。
- 公告暂停：宣布招标过程暂时中断的公告。
- 澄清公告：对招标文件中的具体信息进行澄清或修正的公告。
- 其他：不属于以上任何一类的特殊公告。
        """

# 清空对话
messages = []

messages.append({"role": "system", "content": """
                    """})

messages.append({"role": "user", "content": html})

messages.append({"role": "system", "content": "请按照以下模版格式化提取信息，并将信息添加到数据表中："})

messages.append({"role": "system", "content": template})

response = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=messages,
    # tools=tools,
    top_p=0.7,
    temperature=0.5,
    max_tokens=4096,
)

In [ ]:
# tools = [{
#     "type": "function",
#     "function": {
#         "name": "get_tender_detail",
#         "description": "判断标题是属于什么类型的公告，并插入到数据库。公告类型包括'招标公告'、'资格审核公告'、'公告答疑'、'公告变更'、'公告暂停'、'澄清公告'、'其他'，每个属性的值为布尔值，表示是否匹配，是则ture，否则false",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "招标公告": {"description": "是否为招标公告", "type": "boolean"},
#                 "资格审核公告": {"description": "是否为资格审核公告", "type": "boolean"},
#                 "公告答疑": {"description": "是否为公告答疑", "type": "boolean"},
#                 "公告变更": {"description": "是否为公告变更", "type": "boolean"},
#                 "公告暂停": {"description": "是否为公告暂停", "type": "boolean"},
#                 "澄清公告": {"description": "是否为澄清公告", "type": "boolean"},
#                 "其他": {"description": "是否为其他类型公告", "type": "boolean"}
#                 },
#             "required": [
#                 '招标公告',
#                 '资格审核公告',
#                 '公告答疑',
#                 '公告变更',
#                 '公告暂停',
#                 '澄清公告',
#                 '其他'
#                 ]
#             }
#         }
#     }]


In [180]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="e6af334544b37a85e83900a9152eb9a0.GzjnBTUhPOhDDJhx")

def tender_categorization(title):
    tools = [{
        "type": "function",
        "function": {
            "name": "get_tender_detail",
            "description": "判断标题是属于什么类型的公告，并插入到数据库。",
            "parameters": {
                "type": "object",
                "properties": {
                    "公告类型": {"description": "公告类型只能是'招标公告'、'资格审核公告'、'答疑公告'、'变更公告'、'澄清公告'和'其他'其中一个，且是唯一类型", "type": "string"}
                    },
                "required": [
                    '公告类型',
                    ]
                }
            }
        }]


    # 清空对话
    messages = []

    messages.append({"role": "system", "content": """
                    公告类型:
                    - 招标公告：正式邀请符合资格的投标人参与投标的公告，提供拟采购的货物、工程或工程服务的基本信息、投标条件等。
                    - 资格审核公告：在招标公告发布前，对潜在投标人的资格条件进行审查的过程，旨在确保只有满足条件的投标人能参与投标。
                    - 答疑公告：对潜在投标人提出的问题给出的官方回答；若无人提问,则公告无答疑。
                    - 变更公告：对先前发布的招标公告内容进行的正式更改,如更改投标截止日期、更改招标文件以及招标的暂停或终止等。
                    - 澄清公告：对招标文件中的具体信息进行澄清或修正的公告；若无人提问,则公告无澄清。
                    - 其他：不属于以上任何一类的时候。
                    """})

    messages.append({"role": "user", "content": "请基于以下公告名称，来判断公告类型，并将类型插入数据库："})

    messages.append({"role": "user", "content": title})

    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=messages,
        top_p=0.7,
        temperature=0.8,
        tools=tools,
        max_tokens=4096,
    )
    return response.choices[0].message.tool_calls[0].function.arguments

In [96]:
print(response.choices[0].message.tool_calls[0].function.arguments)

{"公告类型":"招标公告"}


In [100]:
type(response.choices[0].message.tool_calls[0].function.arguments)

str

In [104]:
response.choices[0].message.tool_calls[0].function.model_dump()['arguments']

'{"公告类型":"招标公告"}'

In [84]:
print(response.choices[0].message.content)

None


In [151]:
from database_util import TENDER_DETAIL_HTML_TABLE_NAME,TENDER_INFO_TABLE_NAME
# def get_cleaned_html(cursor,tender_id):
#     select_query = f"""
#     SELECT cleaned_detail_html,original_detail_html,title
#     FROM {TENDER_DETAIL_HTML_TABLE_NAME}
#     WHERE tender_id = {tender_id}
#     """
#     cursor.execute(select_query)
#     return cursor.fetchall()


def get_tender_title(cursor):
    select_query = f"""
    SELECT id, title
    FROM {TENDER_INFO_TABLE_NAME}
    WHERE id NOT IN (
        SELECT tender_id
        FROM announcement_labels
    )
    """
    cursor.execute(select_query)
    return cursor.fetchall()

In [152]:
from database_util import connect_db

db = connect_db()
cursor = db.cursor()



In [154]:
def get_announcement_types(cursor):
    select_query = "SELECT id, announcement_type FROM announcement_catalog"
    cursor.execute(select_query)
    results = cursor.fetchall()
    return {type: id for id, type in results}

def insert_into_announcement_labels(cursor, tender_id, type_id):
    insert_query = f"INSERT INTO announcement_labels (tender_id, type_id) VALUES ({tender_id}, {type_id})"
    cursor.execute(insert_query)    


In [159]:
import json
# 在程序开始时获取最新的类型列表
announcement_types = get_announcement_types(cursor)

# 当 tender_categorization 返回一个类型时
titles = get_tender_title(cursor)
total_titles = len(titles)
for i, title in enumerate(titles):
    try:
        print(f"Processing title {i+1} of {total_titles}")
        type_temp = tender_categorization(title[1])
        tender_title_type = json.loads(type_temp.replace("'", "\""))
        announcement_type = tender_title_type['公告类型']

        # 获取相应的 id
        type_id = announcement_types[announcement_type]

        # 插入到 announcement_labels 表中
        insert_into_announcement_labels(cursor, title[0], type_id)
    except Exception as e:
        print(f"Error processing title {title[0]}: {e}, title is {title[1]}")
        continue

# 在所有插入操作完成后提交事务
db.commit()

Processing title 1 of 4
Processing title 2 of 4
Processing title 3 of 4
Processing title 4 of 4


In [175]:
def get_titles_with_other_type(cursor):
    # 从数据库中获取类型为 "其他" 的标题
    cursor.execute("""
        SELECT a.tender_id, t.title 
        FROM announcement_labels AS a
        JOIN tender_index AS t ON a.tender_id = t.id
        WHERE a.type_id = (SELECT id FROM announcement_catalog WHERE announcement_type = '其他')
    """)
    return cursor.fetchall()

def reprocess_and_update_titles(cursor, db, titles, announcement_catalog):
    for title in titles:
        try:
            # 重新处理标题
            type_temp = tender_categorization(title[1])
            tender_title_type = json.loads(type_temp.replace("'", "\""))
            announcement_type = tender_title_type['公告类型']

            # 如果类型不再是 "其他"，则更新数据库
            if announcement_type != '其他':
                type_id = announcement_catalog[announcement_type]
                print(f"type_id: {type_id}, title[0]: {title[0]}")  # 打印 type_id 和 title[0] 的值
                cursor.execute("UPDATE announcement_labels SET type_id = %s WHERE tender_id = %s", (type_id, title[0]))
                print(f"Title '{title[1]}' was updated. New type is '{announcement_type}'")
            else:
                print(f"Title '{title[1]}' was not updated. Type remains '其他'")
        except Exception as e:
            print(f"Error reprocessing title {title[0]}: {e}, title is {title[1]}")
            continue

    # 在所有更新操作完成后提交事务
    db.commit()

In [181]:

# 使用 get_titles_with_other_type 函数获取类型为 "其他" 的标题
titles_with_other_type = get_titles_with_other_type(cursor)

# 使用 reprocess_and_update_titles 函数重新处理这些标题并更新数据库
reprocess_and_update_titles(cursor, db, titles_with_other_type, announcement_types)

Title '东莞市供水设施更新改造项目-东莞市供水管网更新改造二期工程（虎门、厚街标段）监理结果公告' was not updated. Type remains '其他'
Title '年产食品塑料包装20万件项目监理无答疑说明及澄清文件' was not updated. Type remains '其他'
